### Multi-Tool Calling with MAF and Azure AI Agent Service

In [ ]:
%pip install agent-framework==1.0.0b251001 python-dotenv azure-ai-projects==1.1.0b4 jsonref==1.1.0

### Setting up the Environment

In [1]:
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential

load_dotenv()
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model = os.getenv("AI_FOUNDRY_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

Project Endpoint:  https://carbonopsdevai4434735199.services.ai.azure.com/api/projects/demo-proj
Model:  gpt-4.1-2


### Loading the Code Interpreter Tool

In [2]:
from agent_framework import AgentRunResponse, ChatResponseUpdate, HostedCodeInterpreterTool

code_interpreter_tool = HostedCodeInterpreterTool()


### Loading the Hosted MCP Tool

In [3]:
from agent_framework import HostedMCPTool

ms_learn_mcp_tool = HostedMCPTool(
    name = "Microsoft Learn MCP Tool",
    url = "https://learn.microsoft.com/api/mcp"
)

### Creating the Agent with Multiple Tools in Azure AI Agent Service 

In [4]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential

# Cell 2: Define async workflow
async def create_multi_tool_agent():
    # Create async Azure credential
    credential = AzureCliCredential()
    
    # Initialize the Azure AI Agent Client
    chat_client = AzureAIAgentClient(async_credential=credential,
                                     project_endpoint=project_endpoint,
                                     model_deployment_name=model)

    try:

        # Create the Multi-Tool Agent
        agent = chat_client.create_agent(
            name="multi-tool-agent",
            instructions="You are a helpful multi-tool agent.",
            tools=[code_interpreter_tool, ms_learn_mcp_tool]
        )

        print("✅ Agent created successfully!")
        return agent

    finally:
        # Clean up async clients
        await chat_client.close()
        await credential.close()


In [5]:
# Cell 3: Run the async function
agent = await create_multi_tool_agent()


✅ Agent created successfully!


In [ ]:
from agent_framework import ChatMessage

async def handle_approvals_with_thread(query: str, agent, thread):
    result = await agent.run(query, thread=thread, store=True)
    while len(result.user_input_requests) > 0:
        new_input = []
        for user_input_needed in result.user_input_requests:
            new_input.append(
                ChatMessage(
                    role="user",
                    contents=[user_input_needed.create_response(True)],
                )
            )
        result = await agent.run(new_input, thread=thread, store=True)
    return result


### Running Different Queries on the Agent

In [10]:
thread = agent.get_new_thread()
query = "Can you write a Python function to calculate the factorial of a number using recursion and give me the output for number 100"
response = await handle_approvals_with_thread(query, agent, thread)

print("\n🧠 Assistant:\n", response)



🧠 Assistant:
 Absolutely! Here’s a Python function that calculates the factorial of a number using recursion:

```python
def factorial(n):
    if n == 0 or n == 1:
        return 1
    return n * factorial(n - 1)
```

Let me calculate the factorial of 100 using this function.The factorial of 100 is:
```
93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000
```


In [11]:
thread = agent.get_new_thread()
query = "How to create an Azure storage account using az cli?"
response = await handle_approvals_with_thread(query, agent, thread)

print("\n🧠 Assistant:\n", response)



🧠 Assistant:
 To create an Azure Storage Account using the Azure CLI (`az cli`), follow these steps:

### 1. **Login to Azure**
```sh
az login
```

### 2. **Set the Subscription (optional)**
If you have more than one subscription:
```sh
az account set --subscription "<your-subscription-name-or-id>"
```

### 3. **Create a Resource Group (if you don’t already have one)**
```sh
az group create --name <resource-group-name> --location <location>
```
Example:
```sh
az group create --name myResourceGroup --location eastus
```

### 4. **Create the Storage Account**
```sh
az storage account create \
  --name <storage-account-name> \
  --resource-group <resource-group-name> \
  --location <location> \
  --sku Standard_LRS
```
- `<storage-account-name>` must be **globally unique** and 3–24 lowercase letters and numbers.
- Common SKUs: `Standard_LRS`, `Standard_GRS`, `Standard_ZRS`, etc.

Example:
```sh
az storage account create \
  --name mystorageacct123 \
  --resource-group myResourceGroup \
 